<a href="https://colab.research.google.com/github/eleazarpsmr/notebook-colab/blob/main/Model_EPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>